In [1]:
import pandas as pd
import tempfile
import json
import datetime
import sys
import os

class MyClass:
    fileDir='/tmp'
    def __init__(self,file=""):
        #creates own file if file is not specified
        if not file:
            self.file=tempfile.NamedTemporaryFile(suffix='.csv',dir=self.fileDir,delete=False).name
            print(self.file)
            print('No Input File Specified Creating Own, Please use '+self.file+' file for read')
        else:
            self.file=file

    def _create(self,key,value,_tot=0):
        print(key,value,_tot)
        key=key.upper()[0:32]
        try:
            size=os.stat(self.file).st_size
            #checks if the file size exceeds 1GB
            if size >(1024*1024*1024):
                print('Data File Exceeds the Limit... Please choose New ..'+ self.file)
                return None
            else:
                print('Data File size is ok ..'+str(size))
        except (pd.errors.EmptyDataError, FileNotFoundError):
            pass
        except PermissionError:
            print('Permission Denied')
            return
            
        try :
            df=pd.read_csv(self.file,sep=',' ,names=['key','value','_cts','_tot'],index_col=0,header=None)
        except (pd.errors.EmptyDataError, FileNotFoundError):
            df=pd.DataFrame(columns=['key','value','_cts','_tot'])
        except PermissionError:
            print('Permission Denied')
            return
        if key not in list(df.index):
            #sets timestamp along with key and value while inserting file
            df=pd.DataFrame([(key,json.dumps(value),datetime.datetime.now().strftime("%s"),_tot)],columns=['key','value','_cts','_tot'])
            try: 
                df.to_csv(self.file, mode='a',index=None,header=None)
                print('Created key Value Pair for .....'+key)
            except PermissionError:
                print('Permission Denied')
        else:
            print('Key Already Exists  ....'+key)


    def _read(self,key):
        key=key.upper()[0:32]
        try:
            df=pd.read_csv(self.file,sep=',' ,names=['key','value','_cts','_tot'],index_col=0,header=None)
        except FileNotFoundError:
            print('File Not Found')
            return None
        if key in list(df.index):
            #checks whether key is available for read based on timetoliveproperty
            if (df.loc[key]['_tot']+df.loc[key]['_cts']>=int(datetime.datetime.now().strftime("%s")) or df.loc[key]['_tot']==0):
                print('Value stored '+df.loc[key]['value'])
                return json.loads(df.loc[key]['value'])
            else:
                print('Value stored is Expired',df.loc[key]['_tot']+df.loc[key]['_cts'],int(datetime.datetime.now().strftime("%s")))
        else:
            print('Key Not Found for Reading ....'+key)

    def _delete(self,key):
        key=key.upper()[0:32]
        try:
            df=pd.read_csv(self.file,sep=',' ,names=['key','value','_cts','_tot'],index_col=0,header=None)
        except FileNotFoundError:
            print('File Not Found')
            return None
        if key in list(df.index):
            #checks whether key is available for read based on timetoliveproperty
            if (df.loc[key]['_tot']+df.loc[key]['_cts']>=int(datetime.datetime.now().strftime("%s"))or df.loc[key]['_tot']==0):
                df=df.loc[set(df.index) - set([key])]
                df.to_csv(self.file, mode='w',header=None)
                print('Deleted Key ...'+key)
            else:
                print('Value stored is Expired',df.loc[key]['_tot']+df.loc[key]['_cts'],int(datetime.datetime.now().strftime("%s")))
        else:
            print('Key Not Found for Deletion .....'+key)


#c=MyClass(file='/tmp/data.csv')
#key='ddddfddd'
#value=json.loads('{"New":"Test3"}')
#c._create(key,value)
#c._read(key)
#c._delete(key)


###Myclass creates a file when specified or it will create its own###

In [2]:
c=MyClass()

/tmp/tmpdv7kzuvn.csv
No Input File Specified Creating Own, Please use /tmp/tmpdv7kzuvn.csv file for read


In [3]:
c=MyClass(file='/tmp/data5.csv')

In [4]:
key='ddddfddd'
value=json.loads('{"New":"Test3"}')

In [5]:
c._create(key,value)

ddddfddd {'New': 'Test3'} 0
Created key Value Pair for .....DDDDFDDD


###when already exisiting key is used it throws an error that key already exists###

In [6]:
c._create(key,value)

ddddfddd {'New': 'Test3'} 0
Data File size is ok ..45
Key Already Exists  ....DDDDFDDD


Read will print the value stored at key

In [7]:
c._read(key)

Value stored {"New": "Test3"}


{'New': 'Test3'}

In [8]:
key='AfjdjhfDSFsdfkjkhsdodfhsdidhfdsfadsfe'
value=json.loads('{"New":"Test4"}')

In [9]:
c._create(key,value)

AfjdjhfDSFsdfkjkhsdodfhsdidhfdsfadsfe {'New': 'Test4'} 0
Data File size is ok ..45
Created key Value Pair for .....AFJDJHFDSFSDFKJKHSDODFHSDIDHFDSF


###It will consider only first 32 characters for the key if key size exceeds 32chars###

In [10]:
c._read(key)

Value stored {"New": "Test4"}


{'New': 'Test4'}

###Delete function deletes the key###

In [11]:
c._delete(key)

Deleted Key ...AFJDJHFDSFSDFKJKHSDODFHSDIDHFDSF


In [12]:
c._read("ffff")

Key Not Found for Reading ....FFFF


In [14]:
key='ddddfddd'
c._read(key)

Value stored {"New": "Test3"}


{'New': 'Test3'}

In [15]:
key='Afjdjhdsfadsfe'
value=json.loads('{"New":"Test4"}')
c._create(key,value)

Afjdjhdsfadsfe {'New': 'Test4'} 0
Data File size is ok ..45
Created key Value Pair for .....AFJDJHDSFADSFE


In [16]:
key='Afjdjhdsfdfsafadsfe'
value=json.loads('{"New":"Test4"}')
c._create(key,value)

Afjdjhdsfdfsafadsfe {'New': 'Test4'} 0
Data File size is ok ..96
Created key Value Pair for .....AFJDJHDSFDFSAFADSFE


In [17]:
key='Afjdjhdsfadsfe'
value=json.loads('{"New":"Test7"}')
c._create(key,value)

Afjdjhdsfadsfe {'New': 'Test7'} 0
Data File size is ok ..152
Key Already Exists  ....AFJDJHDSFADSFE


In [ ]:
key='ZZZZZZZZZZZZZz'
value=json.loads('{"New":"Test4"}')
c._create(key,value)